### This Notebook displays the cleaning transformations and querying operations performed on a spark cluster in Databricks

This task was first done by using some basic EDA such as checking for duplicates, null values, taking value counts and using.describe() etc...

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_replace


display(df_pin)


df_pin = df_pin.withColumn(
    "description",
    when(
        (df_pin.description == "Untitled") |
        (df_pin.description == "No description available") |
        (df_pin.description == "No description available Story format"),
        "None"
    ).otherwise(df_pin.description)
)


df_pin = df_pin.withColumn(
    "tag_list",
    when(
        (df_pin.tag_list == "N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e")|
        (df_pin.tag_list == "0"),
        "None"
    ).otherwise(df_pin.tag_list)
)


df_pin = df_pin.withColumn(
    "title",
    when(
        (df_pin.title == "No Title Data Available"),
        "None"
    ).otherwise(df_pin.title)
)


df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
df_pin = df_pin.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))


df_pin = df_pin.withColumn(
    "follower_count",
    when(
        (df_pin.follower_count == "User Info Error"),
        "0"
    ).otherwise(df_pin.follower_count)
)


df_pin = df_pin.withColumn(
    "image_src",
    when(
        (df_pin.image_src == "Image src error."),
        "None"
    ).otherwise(df_pin.image_src)
)


df_pin = df_pin.withColumn(
    "poster_name",
    when(
        (df_pin.poster_name == "User Info Error"),
        "None"
    ).otherwise(df_pin.poster_name)
)


df_pin = df_pin.withColumn(
    "downloaded",
    when(
        (df_pin.downloaded == "None"),
        "0"
    ).otherwise(df_pin.downloaded)
)


# change the datatype of the "follower_count" column to int
df_pin = df_pin.withColumn("follower_count", col("follower_count").cast("int"))


df_pin = df_pin.withColumnRenamed("index", "ind")


df_pin = df_pin.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))


df_pin = df_pin.select(col("ind"), col("unique_id"), col("title"), col("description"), col("follower_count"), col("poster_name"), col("tag_list"), col("is_image_or_video"), col("image_src"), col("save_location"), col("category"))

In [ ]:
from pyspark.sql.functions import array

display(df_geo)


df_geo = df_geo.withColumn("coordinates", array("latitude", "longitude"))


df_geo = df_geo.drop("latitude", "longitude")


# change the datatype of the "timestamp" column to timestamp
df_geo = df_geo.withColumn("timestamp", col("timestamp").cast("timestamp"))


df_geo = df_geo.select(col("ind"), col("country"), col("coordinates"), col("timestamp"))

In [ ]:
from pyspark.sql.functions import concat


display(df_user)


df_user = df_user.withColumn('user_name', concat(df_user.first_name, df_user.last_name))


df_user = df_user.drop("first_name", "last_name")


# change the datatype of the "date_joined" column to timestamp
df_user = df_user.withColumn("date_joined", col("date_joined").cast("timestamp"))


df_user = df_user.select(col("ind"), col("user_name"), col("age"), col("date_joined"))

In [ ]:
#### 1. Find the most popular category people post to based on their country.


In [ ]:
from pyspark.sql.functions import count, desc, struct

df_most_popular_category_by_country = df_pin.join(df_geo, on='ind') \
       .groupBy('country', 'category') \
       .agg(count('*').alias('count')) \
       .groupBy('country') \
       .agg(max(struct('count', 'category')).alias('max_count')) \
       .select('country', 'max_count.category', 'max_count.count') \
       .withColumnRenamed("count", "category_count")

df_most_popular_category_by_country.show()

In [ ]:
#### 2. Find which was the most popular category each year

In [ ]:
from pyspark.sql.functions import year

# Join df_pin with df_geo on the 'ind' column and filter by timestamp range
df_most_popular_category_by_year = df_pin.join(df_geo, on='ind') \
    .filter((year('timestamp') >= 2018) & (year('timestamp') <= 2022))

# Group by year and category, and count the number of occurrences
df_most_popular_category_by_year = df_most_popular_category_by_year.groupBy(year('timestamp').alias('post_year'), 'category') \
       .agg(count('*').alias('category_count'))

# Find the most popular category for each year
df_most_popular_category_by_year = df_most_popular_category_by_year.orderBy(['post_year', 'category_count'], ascending=[True, False]) \
    .groupBy('post_year') \
    .agg({'category': 'first', 'category_count': 'first'}) \
    .withColumnRenamed('first(category)', 'category') \
    .withColumnRenamed('first(category_count)', 'category_count')

# Show the result
df_most_popular_category_by_year.show()

In [ ]:
#### 3. Find the user with the most followers in each country

In [ ]:
from pyspark.sql.functions import max, rank
from pyspark.sql.window import Window

# Find the user with the most followers in each country
df_most_followers_by_country = df_pin.join(df_geo, on='ind') \
      .groupBy('country', 'poster_name') \
      .agg(max('follower_count')) \
      .withColumn('rank', rank().over(Window.partitionBy('country').orderBy(desc('max(follower_count)')))) \
      .filter('rank = 1') \
      .select('country', 'poster_name', 'max(follower_count)') \
      .withColumnRenamed('max(follower_count)', 'follower_count')

df_most_followers_by_country.show()

# Find the country with the user that has the most followers
df_most_followers_country = df_most_followers_by_country.groupBy('country') \
      .agg(max('follower_count').alias('follower_count')) \
      .orderBy(desc('follower_count')) \
      .limit(1) \
      .select('country', 'follower_count')

df_most_followers_country.show()

In [ ]:
#### 4. Find the most popular category for different age groups

In [ ]:
from pyspark.sql.functions import when

df_user_with_age_group = df_pin.join(df_user, on="ind") \
    .withColumn("age_group", 
                when(df_user.age.between(18, 24), "18-24")
                .when(df_user.age.between(25, 35), "25-35")
                .when(df_user.age.between(36, 50), "36-50")
                .when(df_user.age > 50, "+50")
                .otherwise("Unknown"))

df_category_count_by_age = df_user_with_age_group.groupBy("age_group", "category") \
    .agg(count("*").alias("category_count")) \
    .groupBy("age_group") \
    .agg(max(struct("category_count", "category")).alias("max_count")) \
    .select("age_group", "max_count.category", "max_count.category_count") \

df_category_count_by_age.show()

In [ ]:
#### 5. Find the median follower count for different age groups

In [ ]:
from pyspark.sql.functions import percentile_approx

df_median_follower_count_by_age = df_user_with_age_group.groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count")) \
    .select("age_group", "median_follower_count")

df_median_follower_count_by_age.show()

In [ ]:
#### 6. Find how many users have joined each year

In [ ]:
df_user_geo_joined = df_user.join(df_geo, on='ind') \
    .select(year('timestamp').alias('post_year'), 'date_joined') \
    .where("year(timestamp) between 2015 and 2020")

df_users_joined_by_year = df_user_geo_joined.groupBy("post_year") \
    .agg(count("*").alias("number_users_joined"))

df_users_joined_by_year.show()

In [ ]:
#### 7. Find the median follower count of users based on their joining year

In [ ]:
# filter users who joined between 2015 and 2020
df_user_joined = df_user.filter(df_user.date_joined.between('2015-01-01', '2020-12-31'))

# extract the year from the timestamp column in the df_geo table
df_geo_year = df_geo.withColumn("post_year", year("timestamp"))

# join df_user_joined with df_geo_year and df_pin to get follower count for each user
df_join_pin = df_user_joined.join(df_geo_year, on="ind").join(df_pin, on="ind")

# calculate the median follower count for each year
df_median_followers_by_post_year = df_join_pin.groupBy("post_year").agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count"))

# display the results
df_median_followers_by_post_year.show()

In [ ]:
#### 8. Find the median follow count of users based on their joining year and age group

In [ ]:
# Filter users joined between 2015 and 2020
df_user_filtered = df_user.filter((df_user.date_joined >= "2015-01-01") & (df_user.date_joined < "2021-01-01"))

# Join the required dataframes
df_join2 = df_pin.join(df_user_filtered, on = "ind").join(df_geo.select("ind", year("timestamp").alias("post_year")), on="ind").join(df_user_with_age_group.select("age_group", "ind"), on="ind")

# Calculate the median follower count by age group and post year
df_median_follower_by_year_joined = df_join2.groupBy("age_group", "post_year").agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count"))

# Print the results
df_median_follower_by_year_joined.show()